## Classificação de variáveis

### Importando as bibliotecas 

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from slugify import slugify
from sklearn.model_selection import train_test_split
import Levenshtein
#from iowalicor.data.preprocessing import slugify
#from application.app.folder.file import func_name
pd.set_option('display.max_rows', 500)

In [112]:
df_firstn = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv", nrows=10)
df_firstn

,Unnamed: 0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,2364808,S10798700028,02/26/2013,4347,Jeff's Market / Blue Grass,102 W MAYNE,BLUE GRASS,52726,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",82.0,...,22155,Wild Turkey 101 Round,12,375,$6.00,$8.99,1,$8.99,0.38,0.10
1,12139684,INV-06802100005,08/22/2017,3651,Wal-Mart 1491 / Indianola,1500 North Jefferson,Indianola,50125,1500 North Jefferson\nIndianola 50125\n,91.0,...,11298,Crown Royal Canadian Whisky,6,1750,$31.99,$47.99,1,$287.94,1.75,0.46
2,31671,S14182900098,08/28/2013,2576,Hy-Vee Wine and Spirits / Storm Lake,1250 N LAKE ST,STORM LAKE,50588,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",11.0,...,86688,Firefly Strawberry Moonshine,12,750,$9.88,$14.82,2,$29.64,1.50,0.40
3,1100266,S11292900026,03/26/2013,3756,Main Street Liquors / Manning,316 MAIN ST,MANNING,51455,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",14.0,...,69947,Rumple Minze Peppermint Schnapps Liqueur,12,1000,$15.15,$22.72,1,$22.72,1.00,0.26
4,12120532,INV-06737700004,08/17/2017,4997,Downtown Pantry,"218, 6th Ave #101",Des Moines,50309,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",77.0,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,$11.50,$17.25,1,$17.25,0.50,0.13
5,11670503,INV-05415200028,06/08/2017,2595,Hy-Vee Wine and Spirits / Denison,"1620 4th Ave, South",Denison,51442,"1620 4th Ave, South\nDenison 51442\n(42.012395...",24.0,...,10828,Crown Royal Northern Harvest Rye,12,750,$17.50,$26.25,2,$26.25,1.50,0.39
6,4543410,S11437200032,04/03/2013,3712,Monte Spirits,109 N 4TH ST,MONTEZUMA,50171,"109 N 4TH ST\nMONTEZUMA 50171\n(41.585429, -92...",79.0,...,77776,Wild Turkey American Honey,12,750,$10.50,$15.75,2,$31.50,1.50,0.40
7,8212094,S29576500047,12/10/2015,2671,Hy-Vee / Jefferson,"106, W WASHINGTON ST",JEFFERSON,50129,"106, W WASHINGTON ST\nJEFFERSON 50129\n(42.017...",37.0,...,10807,Crown Royal Regal Apple,12,750,$15.00,$22.50,60,$1350.00,45.00,11.89
8,1665619,S20509100146,08/06/2014,3495,Great Pastimes,228 N MAIN ST,MONTICELLO,52310,"228 N MAIN ST\nMONTICELLO 52310\n(42.240132, -...",53.0,...,89194,Jose Cuervo Especial Reposado Flask,24,375,$5.50,$8.25,3,$24.75,1.12,0.30
9,1800759,S10087900002,01/16/2013,2535,Hy-Vee Food Store #1 / WDM,1700 VALLEY WEST DR,WEST DES MOINES,50265,1700 VALLEY WEST DR\nWEST DES MOINES 50265\n,77.0,...,27290,Iowa Shine,12,375,$10.08,$15.13,12,$181.56,4.50,1.19


## Análise exploratória dataset Iowa_Liquor_Sales

In [113]:
#req_cols=['Date','City','County','Item Description','State Bottle Cost','State Bottle Retail','Volume Sold (Liters)']

### Leitura do dataset gerado pelo train_test_split e atribuindo a uma variável

In [114]:
#dataset = pd.read_csv("../data/raw/Iowa_Liquor_Sales.csv", usecols=req_cols)
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv",dtype={
    'Invoice/Item Number': str,
    'Date':str,
    'Store Number': float,
    'Store Name': str,
    'Address': str,
    'City': str,
    'Zip Code': str,
    'Store Location': str,
    'County Number': float,
    'County': str,
    'Category': float,
    'Category Name': str,
    'Vendor Number': float,
    'Vendor Name': str,
    'Item Number': float,
    'Item Description': str,
    'Pack': float,
    'Bottle Volume (ml)': int,
    'State Bottle Cost': str,
    'State Bottle Retail': str,
    'Bottles Sold': int,
    'Sale (Dollars)': str,
    'Volume Sold (Liters)': float,
    'Volume Sold (Gallons)': float
})

In [115]:
df.head(2)

,Unnamed: 0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,2364808,S10798700028,02/26/2013,4347.0,Jeff's Market / Blue Grass,102 W MAYNE,BLUE GRASS,52726,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",82.0,...,22155.0,Wild Turkey 101 Round,12.0,375,$6.00,$8.99,1,$8.99,0.38,0.10
1,12139684,INV-06802100005,08/22/2017,3651.0,Wal-Mart 1491 / Indianola,1500 North Jefferson,Indianola,50125,1500 North Jefferson\nIndianola 50125\n,91.0,...,11298.0,Crown Royal Canadian Whisky,6.0,1750,$31.99,$47.99,1,$287.94,1.75,0.46


### Utilização o train_test_split para dividir o dataset em dados

In [116]:
#X = dataset
#y = dataset

In [117]:
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [118]:
#df = pd.DataFrame(X_test)
#df.head()

In [119]:
#df.to_csv('iowa_liquor_train_test_split_sample.csv')

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518216 entries, 0 to 2518215
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   Invoice/Item Number    object 
 2   Date                   object 
 3   Store Number           float64
 4   Store Name             object 
 5   Address                object 
 6   City                   object 
 7   Zip Code               object 
 8   Store Location         object 
 9   County Number          float64
 10  County                 object 
 11  Category               float64
 12  Category Name          object 
 13  Vendor Number          float64
 14  Vendor Name            object 
 15  Item Number            float64
 16  Item Description       object 
 17  Pack                   float64
 18  Bottle Volume (ml)     int64  
 19  State Bottle Cost      object 
 20  State Bottle Retail    object 
 21  Bottles Sold           int64  
 22  Sale (Dollars)    

### Limpeza os dados e facilitação da análise

In [121]:
df.columns = [slugify(col, lowercase=True, separator='_') for col in df.columns]
df.columns

Index(['unnamed_0', 'invoice_item_number', 'date', 'store_number',
       'store_name', 'address', 'city', 'zip_code', 'store_location',
       'county_number', 'county', 'category', 'category_name', 'vendor_number',
       'vendor_name', 'item_number', 'item_description', 'pack',
       'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons'],
      dtype='object')

In [122]:
#df['city'] = slugify(str(df['city']))

### Escolha das colunas a utilizar

In [ ]:
df = df.drop('unnamed_0', 1)
df = df.drop('invoice_item_number', 1)
df = df.drop('volume_sold_gallons', 1)

In [ ]:
df.columns

In [ ]:
df.info()

### Substituição dos valores NaN da tabela

In [ ]:
df.isnull().values.any()

In [ ]:
for x in df.columns:
    df[x] = df[x].fillna('no_data')

In [ ]:
df[df['city'] == 'nan'] = 'no_city'

In [ ]:
df[df['county'] == 'nan'] = 'no_county'

In [ ]:
df[df['item_description'] == 'nan'] = 'no_item_description'

In [ ]:
df.isnull().values.any()

### Aplicão do slugify  nas colunas com string:

In [ ]:
df['store_name']=df['store_name'].apply(slugify)

In [ ]:
df['address']=df['address'].apply(slugify)

In [ ]:
df['city']=df['city'].apply(slugify)

In [ ]:
df['store_location']=df['store_location'].apply(slugify)

In [ ]:
df['county']=df['county'].apply(slugify)

In [ ]:
df['category_name']=df['category_name'].apply(slugify)

In [ ]:
df['vendor_name']=df['vendor_name'].apply(slugify)

In [ ]:
df['item_description']=df['item_description'].apply(slugify)

In [ ]:
df.head()

In [ ]:
#remover valores duplicados
#def distance(s1,s2):
    #if Levenshtein.distance(s1,s2) != 0:
        #return s1

In [ ]:
#cities_slg

In [ ]:
#counties = df['county'].unique().astype(str)

In [ ]:
df['state_bottle_cost'] = df['state_bottle_cost'].replace({'\$':''}, regex = True)

In [ ]:
df['state_bottle_retail'] = df['state_bottle_retail'].replace({'\$':''}, regex = True)

In [ ]:
df['sale_dollars'] = df['sale_dollars'].replace({'\$':''}, regex = True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

## Explorando o dataset:

#### Verificando formato dos dados:

#### Transformando o os dados da coluna Date em formato datetime

### Em qual condado se vendeu mais?

In [ ]:
county_sales = df['county'].value_counts().nlargest(20)
county_sales.plot(kind="barh",figsize=(15,8))

### Em qual condado se vendeu menos?

In [ ]:
county_sales = df['county'].value_counts().nsmallest(20)
county_sales.plot(kind="barh",figsize=(15,8))

### Sabendo que o condado de POLK foi o que vendeu mais, quais cidades do condado de POLK venderam mais?

#### Filtrando as cidades do condado de POLK

In [ ]:
polk = df[df['county']=='polk']
polk

#### Quais cidades há no condado de POLK?

In [ ]:
polk['city'].unique()

#### Em quais cidade do condado de POLK se vendeu mais bebida?

In [ ]:
polk['city'].value_counts().plot(kind="barh",figsize=(15,8))

#### Quais cidades no estado do IOWA venderam mais bebida?

In [ ]:
city_sales = df['city'].value_counts().nlargest(20)
city_sales.plot(kind="barh",figsize=(15,8))

#### Quais cidades no estado do IOWA venderam mais bebida?

In [ ]:
city_sales = df['city'].value_counts().nsmallest(20)
city_sales.plot(kind="barh",figsize=(15,8))

#### Quais as bebida mais vendida?

In [ ]:
scatter_description = df['item_description'].value_counts().nlargest(10).rename_axis('description').reset_index(name='qtt')
plt.barh(scatter_description['description'],scatter_description['qtt'])
#city_sales = df['item_description'].value_counts().nlargest(20)
#city_sales.plot(kind="barh",figsize=(15,8))

### Analise temporal dos dados

#### Qual a data máxima e mínima?

In [ ]:
max(df['date'])

In [ ]:
min(df['date'])

#### 2012

In [ ]:
year_2012 = df[df['date'].dt.year == 2012]
year_2012.head(1)

#### 2013

In [ ]:
year_2013 = df[df['date'].dt.year == 2013]
year_2013.head(1)

#### 2014

In [ ]:
year_2014 = df[df['date'].dt.year == 2014]
year_2014.head(1)

#### 2015

In [ ]:
year_2015 = df[df['date'].dt.year == 2015]
year_2015.head(1)

#### 2016

In [ ]:
year_2016 = df[df['date'].dt.year == 2016]
year_2016.head(1)

#### 2017

In [ ]:
year_2017 = df[df['date'].dt.year == 2017]
year_2017.head(1)

#### Todo o período

In [ ]:
year1 = [2012,2013,2014,2015,2016,2017]
qtt1 = []
qtt1.append(len(year_2012.index))
qtt1.append(len(year_2013.index))
qtt1.append(len(year_2014.index))
qtt1.append(len(year_2015.index))
qtt1.append(len(year_2016.index))
qtt1.append(len(year_2017.index))
qtt1

In [ ]:
plt.grid(True)
plt.plot(year1, qtt1)

In [ ]:
year2 = [2012,2013,2014,2015,2016]
qtt2 = []
qtt2.append(len(year_2012.index))
qtt2.append(len(year_2013.index))
qtt2.append(len(year_2014.index))
qtt2.append(len(year_2015.index))
qtt2.append(len(year_2016.index))
qtt2

In [ ]:
plt.plot(year2, qtt2)
plt.show()

In [ ]:
volume_per_month_2012 = year_2012['date']

In [ ]:
volume_per_month_2013 = year_2013['date']

In [ ]:
volume_per_month_2014 = year_2014['date']

In [ ]:
volume_per_month_2015 = year_2015['date']

In [ ]:
volume_per_month_2016 = year_2016['date']

In [ ]:
volume_per_month_2017 = year_2017['date']

In [ ]:
volume_per_month_2012.groupby(volume_per_month_2012.dt.month).count().plot(label="2012")
plt.legend()

In [ ]:
volume_per_month_2012.groupby(volume_per_month_2012.dt.month).count().max()

In [ ]:
volume_per_month_2013.groupby(volume_per_month_2013.dt.month).count().plot(label="2013")
plt.legend()

In [ ]:
volume_per_month_2014.groupby(volume_per_month_2014.dt.month).count().plot(label="2014")
plt.legend()

In [ ]:
volume_per_month_2015.groupby(volume_per_month_2015.dt.month).count().plot(label="2015")
plt.legend()

In [ ]:
volume_per_month_2016.groupby(volume_per_month_2016.dt.month).count().plot(label="2016")
plt.legend()

In [ ]:
volume_per_month_2017.groupby(volume_per_month_2017.dt.month).count().plot(label="2017")
plt.legend()